Currently the requirements to get IV estimates are:
 - Your player level (which must remain constant as you're leveling this pokémon)
 - The pokémon's CP, HP, and stardust requirements for the next level
 - Every additional line of data added must be from only one power up (i.e. don't power up many times without also including those values in the table
 
 In the near future I will add the ability to:
 - Specify the pokémon's level or a range of levels to add in as a prior
 - Specify whether the pokémon was caught in the "wild" or from an "egg", and whether you've powered it up or not yet 
 - Put in any data and indicate if any power-up levels have been skipped


### What is your player level?

In [4]:
playerLevel = 40

### What is the pokémon's name? (type in quote marks)

In [5]:
pokemonName= "sandshrew"

### First data set! Put in the pokémon's CP, HP, and stardust to level

In [10]:
#data=(232,42,1900) #pidgey
#data=(1095,131,1300) #evil lapras
data=(219,47,1000) #mudball Jnr lv 1
#data=(230,48,1000) #mudball Jnr lv 2
#data=(241,50,1300) #mudball Jnr lv 3
#data=(287,36,1900) #ekans1
#data=(246,35,1300) #ekans2
#data=(174,53,600) #rhyhorn lv 1
#data=(191,55,600) #rhyhorn lv 2
#data=(209,58,600) #rhyhorn lv 3
#data=(226,60,800) #rhyhorn lv 4
#data=(727,83,1300) #Hypno lv 1
#data=(758,85,1300) #Hypno lv 2
#data=(788,87,1600) #Hypno lv 3
#data=(540,60,1300) #Weepingbell lv 1
#data=(563,61,1300) #Weepingbell lv 2
#data=(585,62,1600) #Weepingbell lv 3
data=(237,45,1600) #pidgey lv 1
data=(246,46,1600) #pidgey lv 2
data=(255,47,1600) #pidgey lv 3
data=(264,48,1600) #pidgey lv 4
data=(272,49,1900) #pidgey lv 5
data=(283,49,1900) #pidgey lv 6
data=(292,50,1900) #pidgey lv 7
data=(301,51,1900) #pidgey lv 8 (but can't level)

##### Code is going to chug along now

In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pokedicts as pd

print pd.pokemonToBaseStats
baseSta,baseAtt,baseDef = pd.pokemonToBaseStats[pokemonName]

dataCP,dataHP,dataSD=data

#HP IV calc is easiest
#HP = (basestamina + ivstamina)*TotalCPMultiplier

#CP = MAX(10,FLOOR(HP**0.5*Attack*Def**0.5/10))
#where
#Attack = (baseatt + ivatt)*TotalCPMultiplier
#Defense= (basedef + ivdef)*TotalCPMultiplier
#TCpM =
#CP= MAX(10,FLOOR( (basesta+ivsta)**2 * (basedef+ivatt) * (basedef+ivdef)**2 ) * TCpM**2/10)
#CP=f(ivsta,ivatt,ivdef)TCpM(level)
#HP=h(ivsta)

#I can calculate p(CP|ivsta,ivatt,ivdef,level), p(HP|ivsta,level), p(stardust|level)
#want p(ivsta|HP,CP), p(ivatt|CP), p(ivdef|CP), p(level|stardust,CP,HP) ????

# Try just using stardust for level
# p(level|stardust)=p(stardust|level)*p(level)/p(stardust)
# can immediately restrict p(level) given the player level.


map1d_level=(np.arange(79)+2.)/2.

map1d_Att=np.arange(16)
map1d_Def=np.arange(16)
map1d_Sta=np.arange(16)

#probability of pokemon level
p_level = np.zeros(79)
p_level[map1d_level <= playerLevel]=1
p_level/=np.sum(p_level)

#print p_level

#p_stardust is already set and is again dependent on player level
#p(stardust|player)=p(player|stardust)*p(stardust)/p(player)
#p(player) is a complete given so p(player)=1, equation reduces to
# # p(stardust|player) as this is calculable yay!


#full list of stardusts:
#allPossibleStardusts = np.array([levelToStardust[x] for x in map_level])

#BUT some stardusts are impossible given the player level, mark some as impossible
#allPossibleStardusts[(map_level > playerLevel*2)-1] = -1
#setStardusts = np.sort(np.array(list(set(allPossibleStardusts).remove(-1))))
#
#print allPossibleStardusts
#print setStardusts
#p_stardust = np.array([allPossibleStardusts.count(x) for x in setStardusts],dtype=float)
#p_stardust/=np.float(np.sum(p_stardust))
#
#print p_stardust


#Easiest thing to do would be to calculate all values? 16 x 16 x 16 x 82 pokemon levels?
#That x82 is going to be really inefficient
#Can do 16x16x16 itself and then use matrix multiplication to work out the other

staAttDefLv=np.zeros((16,16,16,44))
#map_staAttDefLv=np.array([mapSta,mapAtt,mapDef,map_level])

atta=map1d_Att+baseAtt
defe=map1d_Def+baseDef
stam=map1d_Sta+baseSta

TCpM= np.array([pd.levelToTCpM[i] for i in map1d_level])

map_sta, map_att, map_def, map_level=np.meshgrid(stam,atta,defe,map1d_level, indexing='ij')

####
#Construct initial prior:
prior_STATDELV=np.ones(map_sta.shape,dtype="float")

#normalise prior
prior_STATDELV/=np.sum(prior_STATDELV)
####



map_TCpM= np.array([pd.levelToTCpM[i] for i in map_level.ravel()]).reshape(map_level.shape)
allPossibleSD=np.array([pd.levelToStardust[i] \
                        for i in map_level.ravel()]).reshape(map_level.shape)


###allPossibleHP =np.floor((map_sta+baseSta)*map_TCpM) ########
allPossibleHP =np.floor(map_sta*map_TCpM) ########
allPossibleHP[allPossibleHP<10]=10

allPossibleCP = np.floor((map_sta**0.5*map_att*map_def**0.5*map_TCpM**2)/10.)
allPossibleCP[allPossibleCP<10]=10


print "min/max possible stardust", np.min(allPossibleSD), np.max(allPossibleSD)
print "min/max possible CP", np.min(allPossibleCP), np.max(allPossibleCP)
print "min/max possible HP", np.min(allPossibleHP), np.max(allPossibleHP)


LH_CP=(allPossibleCP == dataCP) #this is in STATDELV space
LH_HP=(allPossibleHP == dataHP)
LH_SD=(allPossibleSD == dataSD)


LH_Joint=LH_CP*LH_HP*LH_SD ######### REMOVING SD DEPENDENCE
print "\nNumber of possible stamina, attack and defense ivs, and levels: ", np.sum(LH_Joint)
#Fold in the previous posterior as a prior


print "possible iv HP values given HP:",list(set(map_sta[LH_HP]-baseSta))
print "possible iv HP values given SD:",list(set(map_sta[LH_SD]-baseSta))
print "possible levels given HP: ",list(set(map_level[LH_HP]))
print "possible levels given SD: ",list(set(map_level[LH_SD]))

LH_Joint*=prior_STATDELV

#find all cominations of ivs and lvs
possible_STATDELV=np.array([map_sta[LH_Joint]-baseSta,\
                            map_att[LH_Joint]-baseAtt,\
                            map_def[LH_Joint]-baseDef,\
                            map_level[LH_Joint]]).T



print possible_STATDELV

#sortedInd=np.lexsort((T_possible_STATDELV[1],T_possible_STATDELV[0],\
#                      T_possible_STATDELV[2],T_possible_STATDELV[3]))

print "\nAll possible combinations:"
for vals in possible_STATDELV:
    print "level:", vals[3], \
        "ivs: stamina: ", vals[0], \
        "attack: ", vals[1], \
        "defense: ",vals[2]                                             

{'lapras': (260, 186, 190), 'sandshrew': (100, 90, 114), 'rhyhorn': (160, 110, 116), 'ekans': (70, 112, 112), 'pidgey': (80, 94, 90)}
min/max possible stardust 200 12000
min/max possible CP 10.0 798.0
min/max possible HP 10.0 90.0

Number of possible stamina, attack and defense ivs, and levels:  0
possible iv HP values given HP: [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 15]
possible iv HP values given SD: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
possible levels given HP:  [12.5, 11.5, 10.5, 10.0, 11.0, 12.0, 9.5]
possible levels given SD:  [8.0, 8.5, 9.5, 9.0]
[]

All possible combinations:


/Library/Python/2.7/site-packages/ipykernel/__main__.py:122: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.


In [7]:
## Working out probabilities of finding these data values randomly


#Setting up the real CP/HP/SD values, their indices and a dictionary
map1d_CP=np.arange(800,dtype='float')+10
CPtoIndex=dict(zip(map1d_CP,np.arange(len(map1d_CP),dtype='float')))
               
map1d_HP=np.arange(200,dtype='float')+10
HPtoIndex=dict(zip(map1d_HP,np.arange(len(map1d_HP),dtype='float')))

map1d_SD=np.sort(np.array(list(set(np.array(\
                    [levelToStardust[lvl] for lvl in map1d_level],dtype='float')))))
SDtoIndex=dict(zip(map1d_SD, np.arange(len(map1d_SD),dtype='float')))


countsCPHPSD=np.zeros((map1d_CP.shape[0],map1d_HP.shape[0],map1d_SD.shape[0]),dtype="float")

flatCP=allPossibleCP.flatten()
flatHP=allPossibleHP.flatten()
flatSD=allPossibleSD.flatten()


flatCPHPSD=np.array([flatCP,flatHP,flatSD]).T


print "\n\ncalculating bayesian evidence??? (few seconds)"
for CPHPSD in flatCPHPSD:
    #print CPtoIndex[CPHPSD[0]], HPtoIndex[CPHPSD[1]],SDtoIndex[CPHPSD[2]]
    countsCPHPSD[ CPtoIndex[CPHPSD[0]] , HPtoIndex[CPHPSD[1]] , SDtoIndex[CPHPSD[2]] ] +=1

#marginalise
countsCPHPSD/=np.sum(countsCPHPSD)

#Collapse counts down along axes)
countsCP=[np.sum(countsCPHPSD[i,:,:]) for i in range(len(countsCPHPSD[:,0,0]))]
countsHP=[np.sum(countsCPHPSD[:,i,:]) for i in range(len(countsCPHPSD[0,:,0]))]
countsSD=[np.sum(countsCPHPSD[:,:,i]) for i in range(len(countsCPHPSD[0,0,:]))]


plt.plot(map1d_CP,countsCP)
plt.show()

plt.plot(map1d_HP,countsHP)
plt.show()

plt.plot(map1d_SD,countsSD)
plt.show()

NameError: name 'levelToStardust' is not defined